# Web Automation with AWS Bedrock (Claude Haiku 4.5)

This example demonstrates using webtask with AWS Bedrock's Claude Haiku 4.5 model to autonomously control a browser.

## Requirements

- AWS Bedrock API key in `.env` file: `AWS_BEARER_TOKEN_BEDROCK=your-key-here`
- AWS region (default: `us-east-1`)
- boto3 installed: `pip install pywebtask[bedrock]`

In [1]:
# Imports
import os
from dotenv import load_dotenv
from webtask import Webtask
from webtask.integrations.llm import Bedrock
from playwright.async_api import async_playwright

# Load environment variables from .env file
load_dotenv()
print("✅ Environment variables loaded")

✅ Environment variables loaded


/Users/stevewang/Github/web-auto/webtask/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Agent with Bedrock

Create a webtask agent using AWS Bedrock's Claude Haiku 4.5 model.

The Bedrock client will automatically use `AWS_BEARER_TOKEN_BEDROCK` from your environment.

In [2]:
# Create Bedrock LLM (uses Claude Haiku 4.5)
# Credentials are read from AWS_BEARER_TOKEN_BEDROCK environment variable
llm = Bedrock(
    model="us.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name=os.getenv("AWS_REGION", "us-east-1"),
    temperature=0.5,
)

# Launch browser
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

# Create agent (stateful by default - maintains context between tasks)
agent = await Webtask().create_agent_with_browser(llm=llm, browser=browser)

print("🤖 Agent ready with Claude Haiku 4.5!")

🤖 Agent ready with Claude Haiku 4.5!


## Execute Task

Let the agent autonomously perform a web automation task using DuckDuckGo.

In [3]:
# Execute autonomous task
print("🔍 Executing task...\n")

result = await agent.do(
    "Go to duckduckgo.com and search for 'AWS Bedrock Claude Sonnet 4.5'"
)

🔍 Executing task...

2025-11-20 01:48:31 - webtask._internal.agent.task_runner - INFO - Task start - Task: Go to duckduckgo.com and search for 'AWS Bedrock Claude Sonnet 4.5'
2025-11-20 01:48:31 - webtask._internal.agent.task_runner - INFO - Step 1 - Start
2025-11-20 01:48:31 - webtask._internal.agent.task_runner - DEBUG - Sending LLM request...
2025-11-20 01:48:34 - webtask.llm.llm - INFO - Token usage - Input: 1888, Output: 108, Total: 1996
2025-11-20 01:48:34 - webtask._internal.agent.task_runner - INFO - Received LLM response - Tools: ['navigate']
2025-11-20 01:48:34 - webtask._internal.agent.task_runner - INFO - Reasoning: I'll help you navigate to DuckDuckGo and search for 'AWS Bedrock Claude Sonnet 4.5'.

Let me start by navigating to duckduckgo.com:
2025-11-20 01:48:34 - webtask._internal.agent.tool_registry - INFO - Executing tool: navigate with params: {'url': 'https://www.duckduckgo.com'}
2025-11-20 01:48:35 - webtask._internal.agent.tool_registry - INFO - Tool executed succ

In [4]:
print("\n" + "="*60)
print("📊 Task Result:")
print("="*60)
print(f"Status: {result.status}")
print(f"Feedback: {result.feedback}")


📊 Task Result:
Status: Status.COMPLETED
Feedback: Successfully navigated to duckduckgo.com and completed a search for 'AWS Bedrock Claude Sonnet 4.5'. The search results page is now displaying multiple relevant articles and resources about the topic.


## Try Another Task

The agent maintains context and can execute follow-up tasks.

In [5]:
# Follow-up task
result = await agent.do(
    "Click on the first search result and return me a summary of the page content."
)

2025-11-20 01:48:54 - webtask._internal.agent.task_runner - INFO - Task start - Task: Click on the first search result and return me a summary of the page content.
2025-11-20 01:48:54 - webtask._internal.agent.task_runner - INFO - Step 1 - Start
2025-11-20 01:48:54 - webtask._internal.agent.task_runner - DEBUG - Sending LLM request...
2025-11-20 01:48:58 - webtask.llm.llm - INFO - Token usage - Input: 14462, Output: 187, Total: 14649
2025-11-20 01:48:58 - webtask._internal.agent.task_runner - INFO - Received LLM response - Tools: ['click']
2025-11-20 01:48:58 - webtask._internal.agent.task_runner - INFO - Reasoning: I can see the DuckDuckGo search results page for "AWS Bedrock Claude Sonnet 4.5". The first search result is the AWS article titled "Anthropic's Claude Sonnet 4.5 is now in Amazon Bedrock - AWS" from aws.amazon.com dated Sep 29, 2025. I'll click on this first result to navigate to the page.
2025-11-20 01:48:58 - webtask._internal.agent.tool_registry - INFO - Executing tool:

In [6]:
import json

print(f"Status: {result.status}")
print(f"Feedback: {result.feedback}")
print(f"Output: {json.dumps(result.output, indent=2)}")

Status: Status.COMPLETED
Feedback: Successfully clicked on the first search result and accessed the AWS announcement page. The page displays comprehensive information about Claude Sonnet 4.5's availability in Amazon Bedrock, including its capabilities, use cases, new features, and links to relevant resources.
Output: {
  "page_title": "Anthropic's Claude Sonnet 4.5 is now in Amazon Bedrock",
  "posted_date": "Sep 29, 2025",
  "url": "https://aws.amazon.com/about-aws/whats-new/2025/09/anthropics-claude-sonnet-4-5-amazon-bedrock/",
  "summary": {
    "overview": "Customers can now use Claude Sonnet 4.5 in Amazon Bedrock, a fully managed service offering high-performing foundation models from leading AI companies. Claude Sonnet 4.5 is Anthropic's most intelligent model, excelling at complex agents, coding, and long-horizon tasks while maintaining optimal speed and cost-efficiency.",
    "key_capabilities": [
      "Leads SWE-bench Verified benchmarks",
      "Enhanced instruction followin

## Cleanup

Close the browser when done.

In [7]:
# Cleanup
await browser.close()
await playwright.stop()
print("👋 Browser closed.")

👋 Browser closed.
